In [51]:

# === IMPORTS ===
import os
import json
import time
import requests
import pandas as pd
from pathlib import Path
from tqdm import tqdm


In [52]:

# === GROK API setup ===
api_key = "API Key"
api_url = "https://api.x.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}


In [53]:
def create_political_intensity_prompt(summary, dialogue_text):
    return (
        "You are a cultural analyst evaluating how politically themed a film is.\n\n"
        "You will receive:\n"
        "- A movie summary (to understand the main message and themes)\n"
        "- A chunk of dialogue (to analyze tone, values, framing, and subject matter)\n\n"
        "---\n\n"
        "## Your Task:\n\n"
        "Assess the **overall political intensity** of the film on a scale from 0 to 1.\n\n"
        "- **0** = Apolitical (no meaningful political or ideological content)\n"
        "- **0.5** = Contains some light political references or undertones\n"
        "- **1** = Highly political (themes of power, ideology, identity, societal conflict are central)\n\n"
        "You should consider:\n"
        "- The presence of political or ideological issues (e.g., war, race, gender, justice, institutions, activism, class)\n"
        "- Whether the film appears to *comment on*, *critique*, or *promote* political ideas\n"
        "- How central these political themes are to the overall narrative and dialogue\n\n"
        "Do **not** assess whether it is left- or right-leaning — only how *politically themed* it is.\n\n"
        "---\n\n"
        "⚠️ IMPORTANT: Output only the JSON object below. Do not include any explanations, text, or commentary. Only return the JSON.\n\n"
        "### Output Format (strict JSON):\n\n"
        "{\n"
        '  "political_intensity": <float>,\n'
        '  "confidence": <float>\n'
        "}\n\n"
        "---\n\n"
        f"Movie Summary:\n\"\"\"{summary}\"\"\"\n\n"
        f"Dialogue Chunk:\n\"\"\"{dialogue_text}\"\"\""
    )


In [54]:

# === Grok call ===
def call_grok(prompt):
    payload = {
        "model": "grok-3-mini",
        "messages": [
            {
                "role": "system",
                "content": "You are a strict JSON-returning film analyst. Only respond in the precise JSON structure requested."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.0,
        "max_tokens": 1000,
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 429:
        print("⚠️ Rate limit hit. Waiting 60 seconds...")
        time.sleep(60)
        return call_grok(prompt)

    if response.status_code != 200:
        print(f"⚠️ API Error {response.status_code}: {response.text}")
        return None

    try:
        content = response.json()["choices"][0]["message"]["content"]
        return json.loads(content)
    except Exception as e:
        print(f"⚠️ JSON parsing error: {e}")
        return None


In [55]:

# === Subtitle chunking function ===
def chunk_dialogue(subs, chunk_size=5000, overlap=200):
    text_blocks = []
    for line in subs:
        if line.get("text"):
            text_blocks.append(line["text"].strip())
    full_text = " ".join(text_blocks)

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start = end - overlap
    return chunks


In [56]:

# === Paths ===
base_path = Path.home() / "Desktop" / "processed_subs"
subs_dir = base_path
summaries_dir = base_path / "summaries"
prompts_dir = base_path / "prompts_political_intensity"
output_dir = base_path / "scored_political_intensity_grok"

prompts_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# === Load updated match file ===
matches_df = pd.read_csv("final_matches_updated2.csv")
matches_df = matches_df.dropna(subset=["subtitle_filename"])


In [57]:

# === Main Execution Block ===
for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Scoring political intensity with Grok"):
    filename = row["subtitle_filename"]

    json_path = subs_dir / f"{filename}.json"
    summary_path = summaries_dir / f"{filename}_summary.txt"

    if not json_path.exists() or not summary_path.exists():
        print(f"⚠️ Missing files for {filename}. Skipping.")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        subs = json.load(f)

    with open(summary_path, "r", encoding="utf-8") as f:
        summary = f.read().strip()

    chunks = chunk_dialogue(subs, chunk_size=5000, overlap=200)

    for i, chunk_text in enumerate(chunks):
        output_path = output_dir / f"{filename}_chunk{i+1}_political_intensity_grok.json"

        if output_path.exists():
            print(f"⏩ Skipping already scored: {filename} chunk {i+1}")
            continue

        prompt = create_political_intensity_prompt(summary, chunk_text)
        prompt_path = prompts_dir / f"{filename}_chunk{i+1}_prompt_grok.json"

        with open(prompt_path, "w", encoding="utf-8") as f:
            json.dump({"prompt": prompt}, f, indent=2)

        result = call_grok(prompt)

        if result is not None:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=2)
            print(f"✅ Scored: {filename} chunk {i+1}")
        else:
            print(f"❌ Failed scoring: {filename} chunk {i+1}")

        time.sleep(0.5)  # Respect API rate limits


Scoring political intensity with Grok:  44%|▍| 208/478 [00:00<00:00, 1044.03it/s

⏩ Skipping already scored: 2017_Logan chunk 1
⏩ Skipping already scored: 2017_Logan chunk 2
⏩ Skipping already scored: 2017_Logan chunk 3
⏩ Skipping already scored: 2017_Logan chunk 4
⏩ Skipping already scored: 2017_Logan chunk 5
⏩ Skipping already scored: 2017_Logan chunk 6
⏩ Skipping already scored: 2017_Logan chunk 7
⏩ Skipping already scored: 2017_Logan chunk 8
⏩ Skipping already scored: 2017_Logan chunk 9
⏩ Skipping already scored: 1979_10 chunk 1
⏩ Skipping already scored: 1979_10 chunk 2
⏩ Skipping already scored: 1979_10 chunk 3
⏩ Skipping already scored: 1979_10 chunk 4
⏩ Skipping already scored: 1979_10 chunk 5
⏩ Skipping already scored: 1979_10 chunk 6
⏩ Skipping already scored: 1979_10 chunk 7
⏩ Skipping already scored: 1979_10 chunk 8
⏩ Skipping already scored: 1979_10 chunk 9
⏩ Skipping already scored: 1983_Tootsie chunk 1
⏩ Skipping already scored: 1983_Tootsie chunk 2
⏩ Skipping already scored: 1983_Tootsie chunk 3
⏩ Skipping already scored: 1983_Tootsie chunk 4
⏩ Skipp

Scoring political intensity with Grok:  65%|▋| 313/478 [00:00<00:00, 1043.14it/s

⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 1
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 2
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 3
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 4
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 5
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 6
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 7
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 8
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080p.Bluray.DTS-HD.MA.5.1.X264-EVO chunk 9
⏩ Skipping already scored: 2021_Spider-Man_No_Way_Home.2022.1080

Scoring political intensity with Grok:  65%|▋| 313/478 [00:14<00:00, 1043.14it/s

...

Scoring political intensity with Grok: 100%|█▉| 476/478 [29:56<02:02, 61.44s/it]

✅ Scored: 2025_Sinners chunk 1
✅ Scored: 2025_Sinners chunk 2
✅ Scored: 2025_Sinners chunk 3
✅ Scored: 2025_Sinners chunk 4
✅ Scored: 2025_Sinners chunk 5
✅ Scored: 2025_Sinners chunk 6
✅ Scored: 2025_Sinners chunk 7
✅ Scored: 2025_Sinners chunk 8
✅ Scored: 2025_Sinners chunk 9
✅ Scored: 2025_Sinners chunk 10
✅ Scored: 2025_Sinners chunk 11
✅ Scored: 2025_Sinners chunk 12
✅ Scored: 2025_Sinners chunk 13


Scoring political intensity with Grok: 100%|█▉| 477/478 [31:17<01:06, 66.86s/it]

✅ Scored: 2025_Snow.White chunk 1
✅ Scored: 2025_Snow.White chunk 2
✅ Scored: 2025_Snow.White chunk 3
✅ Scored: 2025_Snow.White chunk 4
✅ Scored: 2025_Snow.White chunk 5
✅ Scored: 2025_Snow.White chunk 6
✅ Scored: 2025_Snow.White chunk 7
✅ Scored: 2025_Snow.White chunk 8
✅ Scored: 2025_Snow.White chunk 9


Scoring political intensity with Grok: 100%|██| 478/478 [32:15<00:00,  4.05s/it]


In [74]:
import pandas as pd
import json
from pathlib import Path
import re
from collections import defaultdict

# === Paths ===
base_path = Path.home() / "Desktop" / "processed_subs"
output_dir = base_path / "scored_political_intensity_grok"
film_csv_path = base_path / "film_political_intensity_grok.csv"
yearly_csv_path = base_path / "yearly_political_intensity_grok.csv"

# === Gather Scored JSON Files ===
json_files = list(output_dir.glob("*.json"))
film_scores = defaultdict(list)
yearly_scores = defaultdict(list)

# === Parse JSONs and Aggregate Scores ===
for file_path in json_files:
    match = re.match(r"(\d{4})_(.+?)_chunk\d+_political_intensity_grok\.json", file_path.name)
    if not match:
        continue

    year = int(match.group(1))
    film = match.group(2)

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            score = data.get("political_intensity")
            if score is not None:
                film_key = f"{year}_{film}"
                film_scores[film_key].append(score)
                yearly_scores[year].append(score)
    except json.JSONDecodeError:
        print(f"⚠️ Skipped invalid JSON: {file_path.name}")

# === Create Film-Level DataFrame ===
film_data = [
    {
        "film": film,
        "year": int(film[:4]),
        "avg_political_intensity": sum(scores) / len(scores),
        "chunks_scored": len(scores)
    }
    for film, scores in film_scores.items()
]
df_films = pd.DataFrame(film_data)
df_films.sort_values(by="year", inplace=True)

# === Create Year-Level DataFrame ===
year_data = [
    {
        "year": year,
        "avg_political_intensity": sum(scores) / len(scores),
        "films_scored": len(set(f for f in film_scores if f.startswith(str(year))))
    }
    for year, scores in sorted(yearly_scores.items())
]
df_yearly = pd.DataFrame(year_data)

# === Save as CSV ===
df_films.to_csv(film_csv_path, index=False)
df_yearly.to_csv(yearly_csv_path, index=False)

print("✅ Aggregation complete.")
print(f"📁 Film-level CSV saved to: {film_csv_path}")
print(f"📁 Yearly average CSV saved to: {yearly_csv_path}")


✅ Aggregation complete.
📁 Film-level CSV saved to: /Users/cedricroetheli/Desktop/processed_subs/film_political_intensity_grok.csv
📁 Yearly average CSV saved to: /Users/cedricroetheli/Desktop/processed_subs/yearly_political_intensity_grok.csv
